In [2]:
import numpy as np
import pandas as pd
import wandb
from ezdl.utils.utilities import mlflow_linearize

In [3]:
api = wandb.Api()

In [16]:
from IPython.core.display_functions import display


def get_runs(run_filters, project="pasqualedem/weeddetection"):
    # Project is specified by <entity/project-name>
    runs = api.runs(project, filters=run_filters)

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k,v in run.config.items()
              if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)
    df = pd.DataFrame([{**mlflow_linearize(dc), **mlflow_linearize(ds)} for dc, ds in zip(config_list, summary_list)])
    return df

def remove_hyp(df):
    to_delete = filter(lambda x: x.startswith('hyper_params'), df.columns)
    display(list(to_delete))
    for l in to_delete:
        df.drop(l, inplace=True)

def rename_values(df: pd.DataFrame, vrenamings: dict):
    for column, values in vrenamings.items():
        if column in df.columns:
            df[column] = df[column].replace(values)

In [45]:
renamings = {
    'in_params.dataset.channels': 'Channels',
    'in_params.train_params.optimizer_params.weight_decay': "Weight Decay",
    "in_params.model.params.side_pretrained": "Multispectral Pretraining",
    'in_params.train_params.initial_lr': 'LR',
    'in_params.model.name': 'Model',
    'in_params.model.params.backbone': 'Backbone',
    'in_params.train_params.freeze_pretrained': 'Strategy',
    'in_params.model.params.main_pretrained': 'Used Weights',
    'train_WrapF1': 'Train F1',
    'f1': 'F1',
    'in_params.dataset.batch_size': 'Batch Size',
    'precision': 'Precision',
    'in_params.train_params.loss.params.gamma': 'Gamma',
    'in_params.model.params.fusion_type': 'Fusion',
    'recall': 'Recall'
}
value_renamings = {
    'Strategy': {True: 'Freeze', False: 'Fine-Tuning'},
    'DropPath': {'conv_sum_drop': 'Conv Drop', 'conv_sum': 'Conv', "squeeze_excite": "SE"},
    'Model': {"wd/models/splitlawin": "SplitLawin", "splitlawin": "SplitLawin"}
}
channels_ordering = {
    "G, NDVI, NIR, R, RE" : 1,
    "R, G, NIR, RE": 2,
    "R, G, RE, NIR": 2,
    "R, G, RE": 3,
    "G, NIR, R": 3,
    "R, G, NIR": 4,
    "R, G, NDVI": 5,
    "R, G": 6,
    "CIR": 7,
    "NDVI": 8,

    'Fine-Tuning': 1,
    'Freeze': 2,

    'R': 1,
    'G': 2,
    'RGB': 2.5,
    'G, G, R': 3,
    'R, G, G, G': 4,

    'MiT-B0': 0,
    'MiT-B1': 1,
    'lawin': 0,
    'laweed': 1,
    'doublelawin': 2,
    'doublelaweed': 3,
}

## Lawin

In [6]:
filters = {
    'group': 'Lawin', 'config.in_params.model.params.backbone': 'MiT-B0'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    # 'in_params.model.name',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [7]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.output_channels',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 'hyper_params.dataset_params.return_path',
 'hyper_params.dataset_params.test_folders',
 'hyper_params.dataset_params.train_folders',
 'hyper_params.dataset_params.val_batch_size',
 

In [8]:
runs_df

,in_params.tags,in_params.model.name,in_params.model.params.backbone,in_params.phases,in_params.dataset.root,in_params.dataset.size,in_params.dataset.channels,in_params.dataset.hor_flip,in_params.dataset.ver_flip,in_params.dataset.crop_size,...,007_frame0127.png.masks.ground_truth.path,007_frame0127.png.masks.ground_truth.size,007_frame0127.png.masks.ground_truth._type,007_frame0127.png.masks.ground_truth.sha256,007_frame0127.png.width,007_frame0127.png.format,007_frame0127.png.height,007_frame0127.png.sha256,007_frame0127.png.path,007_frame0127.png.size
0,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[G, NDVI, NIR, R, RE]",True,True,"[256, 256]",...,media/images/mask/007_frame0127.png_289_39962c...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,14ee441eaa69dac0509faa25d878e3e1326862dd5fc6fc...,media/images/007_frame0127.png_289_14ee441eaa6...,86823
1,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[R, G, RE, NIR]",True,True,"[256, 256]",...,media/images/mask/007_frame0127.png_527_39962c...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,83fe2021a0ae2feb36455a99983b01cdd8c1f0e3f43e9e...,media/images/007_frame0127.png_527_83fe2021a0a...,87294
2,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[G, NIR, R]",True,True,"[256, 256]",...,media/images/mask/007_frame0127.png_765_39962c...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,1cd957d0a56461589f836419dc0c8f24f712a70a2dad52...,media/images/007_frame0127.png_765_1cd957d0a56...,80870
3,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,CIR,True,True,"[256, 256]",...,media/images/mask/007_frame0127.png_583_39962c...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,81fbac38de0b450e20925e14a1046ecad27128611f71fe...,media/images/007_frame0127.png_583_81fbac38de0...,81676
4,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,[NDVI],True,True,"[256, 256]",...,media/images/mask/007_frame0127.png_527_39962c...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,cd596b0f014d1138faacdfa34b991cb54180b759aa6e34...,media/images/007_frame0127.png_527_cd596b0f014...,44953
5,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[R, G]",True,True,"[256, 256]",...,media/images/mask/007_frame0127.png_583_39962c...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,bef071e40865bca5fc2f7f6cafee772eb0896f2bc32a5b...,media/images/007_frame0127.png_583_bef071e4086...,62279
6,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[G, NDVI, NIR, R, RE]",True,True,"[256, 256]",...,media/images/mask/007_frame0127.png_681_39962c...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,9100ff54aac385fd1d8f83a92edb531e8f6f7db81fbfc9...,media/images/007_frame0127.png_681_9100ff54aac...,87984
7,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[R, G, RE, NIR]",True,True,"[256, 256]",...,media/images/mask/007_frame0127.png_667_39962c...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,c24dd25d28b413f53be928718a44de448c0989bf64759e...,media/images/007_frame0127.png_667_c24dd25d28b...,84622
8,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,"[G, NIR, R]",True,True,"[256, 256]",...,media/images/mask/007_frame0127.png_681_39962c...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,f653be4528c184fabbc4ed799239f74b3da1448a4198cb...,media/images/007_frame0127.png_681_f653be4528c...,82495
9,"[crop, B0]",lawin,MiT-B0,"[train, test]",dataset/processed/Sequoia,same,CIR,True,True,"[256, 256]",...,media/images/mask/007_frame0127.png_583_39962c...,141,mask,39962cd5bc9f4f0446341d3e6e0c6c37336ddeb2e026a1...,256,png,256,81fbac38de0b450e20925e14a1046ecad27128611f71fe...,media/images/007_frame0127.png_583_81fbac38de0...,81676


In [9]:
projected_runs = runs_df[
    selected_cols
]

In [10]:
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [11]:
selected_runs

,in_params.dataset.channels,in_params.train_params.optimizer_params.weight_decay,in_params.train_params.initial_lr,f1,precision,recall
0,"[G, NDVI, NIR, R, RE]",0.0005,0.0001,0.326794,0.345053,0.465314
1,"[R, G, RE, NIR]",0.0005,0.0001,0.377526,0.705026,0.357784
2,"[G, NIR, R]",0.0005,0.0001,0.357331,0.499817,0.348049
3,CIR,0.0005,0.0001,0.404106,0.428261,0.403309
4,[NDVI],0.0005,0.0001,0.331022,0.338566,0.456207
5,"[R, G]",0.0005,0.0001,0.356745,0.390026,0.434191
6,"[G, NDVI, NIR, R, RE]",0.0000,0.0001,0.338468,0.390821,0.496304
7,"[R, G, RE, NIR]",0.0000,0.0001,0.372336,0.573994,0.355982
8,"[G, NIR, R]",0.0000,0.0001,0.371983,0.444056,0.481801
9,CIR,0.0000,0.0001,0.369274,0.399371,0.358478


In [12]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

/tmp/ipykernel_56297/2431484179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_56297/2431484179.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs[roundings] = selected_runs[roundings].round(3)


In [13]:
idx = selected_runs.groupby(by='Channels')['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by="Channels", key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

/tmp/ipykernel_56297/2220297888.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_runs.sort_values(by="Channels", key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)


In [14]:
selected_runs

,Channels,Weight Decay,LR,F1,Precision,Recall
0,"G, NDVI, NIR, R, RE",0.0005,0.0001,0.327,0.345,0.465
1,"R, G, RE, NIR",0.0005,0.0001,0.378,0.705,0.358
2,"G, NIR, R",0.0005,0.0001,0.357,0.500,0.348
3,CIR,0.0005,0.0001,0.404,0.428,0.403
4,NDVI,0.0005,0.0001,0.331,0.339,0.456
5,"R, G",0.0005,0.0001,0.357,0.390,0.434
6,"G, NDVI, NIR, R, RE",0.0000,0.0001,0.338,0.391,0.496
7,"R, G, RE, NIR",0.0000,0.0001,0.372,0.574,0.356
8,"G, NIR, R",0.0000,0.0001,0.372,0.444,0.482
9,CIR,0.0000,0.0001,0.369,0.399,0.358


In [15]:
best_runs

,Channels,Weight Decay,LR,F1,Precision,Recall
18,"G, NDVI, NIR, R, RE",0.0000,0.0010,0.340,0.370,0.506
19,"R, G, RE, NIR",0.0000,0.0010,0.483,0.484,0.576
14,"G, NIR, R",0.0005,0.0010,0.400,0.543,0.378
23,"R, G",0.0000,0.0010,0.437,0.414,0.623
3,CIR,0.0005,0.0001,0.404,0.428,0.403
4,NDVI,0.0005,0.0001,0.331,0.339,0.456
22,NDVI,0.0000,0.0010,0.331,0.335,0.409


In [ ]:
best_runs.to_csv("../../exps/lawin.csv")

In [ ]:
selected_runs.to_csv("../../exps/lawin-complete.csv")

## Lawin Aug

In [ ]:
filters = {
    'group': 'LawinAugPatience'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by='Channels')['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by="Channels", key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [ ]:
best_runs.to_csv("../../exps/lawin_aug.csv")

In [ ]:
selected_runs.to_csv("../../exps/lawin_aug-complete.csv")

## Lawin Pretrained

In [ ]:
filters = {
    '$or': [{'group': 'LawinPretrained'}, {'group': 'LawinFrozen'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.optimizer_params.weight_decay',
    'in_params.train_params.initial_lr',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/lawin_pretrained.csv")
selected_runs.to_csv("../../exps/lawin_pretrained-complete.csv")

## Focal

In [ ]:
filters = {
    'group': 'Focal'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/focal.csv")
selected_runs.to_csv("../../exps/focal-complete.csv")

## SRR

In [ ]:
filters = {
    'group': 'RandomRotation'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.backbone',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs
selected_runs.fillna("", inplace=True)

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/rotation.csv")
selected_runs.to_csv("../../exps/rotation-complete.csv")

## Double Encoder

In [ ]:
filters = {
    'group': 'DoubleLawin'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
runs_df

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs
selected_runs.fillna("", inplace=True)

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
best_runs.to_csv("../../exps/doublelawin.csv")
selected_runs.to_csv("../../exps/doublelawin-complete.csv")

## Double Encoder Pre-Trained

In [ ]:
filters = {
    'group': 'DoubleLawinPretrained'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'train_WrapF1',
    'f1',
    'precision',
    'recall'
]
roundings = ['Train F1', 'F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.fillna(0, inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs['Gamma'] = selected_runs['Gamma'].astype(float).astype(int)

In [ ]:
selected_runs

In [ ]:
idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
selected_runs.mean()

In [ ]:
selected_runs.groupby(by=['Gamma']).mean()

In [ ]:
selected_runs.to_csv("../../exps/doublelawinpretrained-complete.csv")
best_runs.to_csv("../../exps/doublelawinpretrained.csv")

## Double Encoder Pre-Trained Drop

In [ ]:
filters = {
    'group': 'DLPretrainedDrop'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.loss.params.gamma',
    'train_WrapF1',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall', 'Train F1']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.fillna("", inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [ ]:
selected_runs

In [ ]:
idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
selected_runs.groupby(by=['Gamma']).mean()

In [ ]:
selected_runs.mean()

In [ ]:
selected_runs.to_csv("../../exps/dlpretraineddrop-complete.csv")
best_runs.to_csv("../../exps/dlpretraineddrop.csv")

## RGB Weights

In [ ]:
filters = {
    'group': 'RedEdge'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.main_pretrained',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.fillna(0, inplace=True)
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['Gamma'] = selected_runs['Gamma'].astype(float).astype(int)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Backbone', 'Channels', 'Strategy', 'Used Weights'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [ ]:
selected_runs

In [ ]:
selected_runs.to_csv("../../exps/rgb_weights-complete.csv")

In [ ]:
filters = {
    '$or': [{'group': 'RedEdge'}, {'group': 'RandomRotation'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.params.backbone',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'f1',
]
roundings = ['F1']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs.loc[(selected_runs['Channels'] == 'CIR') & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'RGB'
selected_runs.loc[(selected_runs['Channels'].isin([['NDVI']])) & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'R'
selected_runs.drop(selected_runs[selected_runs['Channels'].isin([['R', 'G']])].index, inplace=True)

In [ ]:
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Backbone', 'Channels', 'Used Weights'],
                          key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Strategy', 'Used Weights'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels', 'Strategy'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
compare_runs = best_runs.merge(best_runs, on=['Channels', 'Strategy'])
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == compare_runs['Used Weights_y']].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == 'G, G, R'].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Used Weights_x'] == 'G'].index, inplace=True)
compare_runs

In [ ]:
complete = best_runs[best_runs['Channels'] == 'R, G, NIR, RE']
complete.rename(columns={'Gamma': 'Gamma_y', 'F1': 'F1_y', 'Used Weights': 'Used Weights_y', 'Backbone': 'Backbone_x'}, inplace=True)
compare_runs = compare_runs.append(complete)
compare_runs.fillna("", inplace=True)
compare_runs

In [ ]:
compare_runs.to_csv("../../exps/rgb_weights.csv")

## Laweed

In [ ]:
filters = {
    'group': 'Laweed'
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels', 'Strategy'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Strategy', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
selected_runs

In [ ]:
best_runs

In [ ]:
selected_runs.to_csv("../../exps/laweed-complete.csv")

In [ ]:
filters = {
    '$and': [
        {'$or': [{'group': 'Laweed'}, {'group': 'RandomRotation'}, {'group': 'RedEdge'}]},
        {'config.in_params.model.params.backbone': 'MiT-B0'}
    ]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.train_params.freeze_pretrained',
    'in_params.model.params.main_pretrained',
    'in_params.model.name',
    'f1',
]
roundings = ['F1']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs
selected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['Gamma'].fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Channels'],
                          key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
selected_runs.loc[(selected_runs['Channels'] == 'CIR') & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'RGB'
selected_runs.loc[(selected_runs['Channels'].isin([['NDVI']])) & selected_runs['Used Weights'].isna(), 'Used Weights'] = 'R'

In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Model'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Model', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
best_runs

In [ ]:
compare_runs = best_runs.merge(best_runs, on=['Channels'])
compare_runs.drop(compare_runs[compare_runs['Model_x'] == compare_runs['Model_y']].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Model_x'] == 'laweed'].index, inplace=True)
compare_runs

In [ ]:
compare_runs.drop(columns=['Model_x', 'Model_y'], inplace=True)
compare_runs

In [ ]:
compare_runs.to_csv("../../exps/laweed.csv")

## Double Laweed

In [ ]:
filters = {
    '$or': [{'group': 'DoubleLaweedPrBatch'}, {'group': 'DoubleLaweedPrDropBatch'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.fusion_type',
    'f1',
    'precision',
    'recall'
]
roundings = ['F1', 'Precision', 'Recall']

In [ ]:
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
# selected_runs = projected_runs[projected_runs['in_params.model.params.backbone'] == 'MiT-B0']
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['DropPath'].fillna('conv_sum', inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['Gamma'].fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
selected_runs

In [ ]:
selected_runs.to_csv("../../exps/doublelaweed-complete.csv")

In [ ]:
filters = {
    '$or': [{'group': 'DoubleLaweedPrBatch'}, {'group': 'DoubleLaweedPrDropBatch'},{'group': 'DLPretrainedDrop'}]
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.train_params.loss.params.gamma',
    'in_params.model.params.fusion_type',
    'in_params.model.name',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [ ]:
selected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
selected_runs['DropPath'].fillna('conv_sum', inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs['Gamma'].fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)

selected_runs[roundings] = selected_runs[roundings].round(3)
selected_runs.sort_values(by=['Channels'],
                          key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)



In [ ]:
idx = selected_runs.groupby(by=['Channels', 'Model'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs.sort_values(by=['Model', 'Channels'], key=lambda x: x.apply(lambda k: channels_ordering[k]), inplace=True)
best_runs[roundings] = best_runs[roundings].round(3)
best_runs

In [ ]:
compare_runs = best_runs.merge(best_runs, on=['Channels'])

In [ ]:
compare_runs

In [ ]:
compare_runs.drop(compare_runs[compare_runs['Model_x'] == compare_runs['Model_y']].index, inplace=True)
compare_runs.drop(compare_runs[compare_runs['Model_x'] == 'doublelaweed'].index, inplace=True)
compare_runs

In [ ]:
compare_runs.to_csv("../../exps/doublelaweed.csv")

# Final results

In [ ]:
filters = {
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters)
remove_hyp(runs_df)

In [ ]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Model', 'Backbone'])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [ ]:
best_runs

In [ ]:
filters = {
}
selected_cols = [
    'in_params.dataset.channels',
    'in_params.model.name',
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters)
remove_hyp(runs_df)

projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Channels'])['F1'].transform(max) == selected_runs['F1']
best_runs_channel = selected_runs[idx]
best_runs_channel[roundings] = best_runs_channel[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)
best_runs_channel

## SplitLawin RedEdge

In [50]:
filters = {
    '$or': [{'group': 'SplitLawin'}, {'group': 'SplitLawinFineTune'}]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.fusion_type',
    "in_params.model.params.side_pretrained",
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-rededge")
remove_hyp(runs_df)

[]

In [51]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [52]:
selected_runs

,in_params.model.name,in_params.model.params.fusion_type,in_params.model.params.side_pretrained,in_params.model.params.backbone,f1
0,wd/models/splitlawin,squeeze_excite,G,MiT-B1,0.857106
1,wd/models/splitlawin,squeeze_excite,G,MiT-B1,0.865151
2,wd/models/splitlawin,squeeze_excite,G,MiT-B1,0.856090
3,wd/models/splitlawin,squeeze_excite,G,MiT-B1,0.865646
4,wd/models/splitlawin,squeeze_excite,G,MiT-B0,0.864240
...,...,...,...,...,...
83,wd/models/splitlawin,squeeze_excite,G,MiT-B1,0.755687
84,wd/models/splitlawin,squeeze_excite,G,MiT-B0,0.740656
85,wd/models/splitlawin,squeeze_excite,G,MiT-B0,0.769373
86,wd/models/splitlawin,squeeze_excite,G,MiT-B0,0.742202


In [ ]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Model', 'Backbone', "Multispectral Pretraining"])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

In [54]:
best_runs

,Model,Fusion,Multispectral Pretraining,Backbone,F1
3,SplitLawin,squeeze_excite,G,MiT-B1,0.866
4,SplitLawin,squeeze_excite,G,MiT-B0,0.864
9,SplitLawin,squeeze_excite,0,MiT-B1,0.856
32,SplitLawin,conv_sum,0,MiT-B0,0.855


### SplitLawin Sequoia

In [46]:
filters = {
    '$or': [{'group': 'SplitLawin'}, {'group': 'SplitLawinFineTune'}]
}
selected_cols = [
    'in_params.model.name',
    'in_params.model.params.fusion_type',
    "in_params.model.params.side_pretrained",
    'in_params.model.params.backbone',
    'f1',
]
roundings = ['F1']
runs_df = get_runs(filters, project="pasqualedem/weedmapping-sequoia")
rem = remove_hyp(runs_df)

['hyper_params.arch_params.schema',
 'hyper_params.arch_params.sync_bn',
 'hyper_params.arch_params.backbone',
 'hyper_params.arch_params.fusion_type',
 'hyper_params.arch_params.in_channels',
 'hyper_params.arch_params.num_classes',
 'hyper_params.arch_params.out_channels',
 'hyper_params.arch_params.main_channels',
 'hyper_params.arch_params.input_channels',
 'hyper_params.arch_params.output_channels',
 'hyper_params.arch_params.side_pretrained',
 'hyper_params.arch_params.backbone_pretrained',
 'hyper_params.dataset_params.root',
 'hyper_params.dataset_params.size',
 'hyper_params.dataset_params.schema',
 'hyper_params.dataset_params.s3_link',
 'hyper_params.dataset_params.channels',
 'hyper_params.dataset_params.hor_flip',
 'hyper_params.dataset_params.ver_flip',
 'hyper_params.dataset_params.crop_size',
 'hyper_params.dataset_params.batch_size',
 'hyper_params.dataset_params.dataset_dir',
 'hyper_params.dataset_params.num_classes',
 'hyper_params.dataset_params.num_workers',
 'hyp

In [47]:
projected_runs = runs_df[
    selected_cols
]
selected_runs = projected_runs

In [48]:
selected_runs.rename(columns=renamings, inplace=True)
rename_values(selected_runs, value_renamings)
selected_runs[roundings] = selected_runs[roundings].round(5)
selected_runs.fillna(0, inplace=True)
selected_runs = selected_runs.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
idx = selected_runs.groupby(by=['Model', 'Backbone', "Multispectral Pretraining"])['F1'].transform(max) == selected_runs['F1']
best_runs = selected_runs[idx]
best_runs[roundings] = best_runs[roundings].round(3)
selected_runs[roundings] = selected_runs[roundings].round(3)

/tmp/ipykernel_56297/2600791792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_runs.rename(columns=renamings, inplace=True)
/tmp/ipykernel_56297/1251592377.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].replace(values)
/tmp/ipykernel_56297/2600791792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [49]:
best_runs

,Model,Fusion,Multispectral Pretraining,Backbone,F1
12,SplitLawin,conv_sum,0,MiT-B1,0.571
18,SplitLawin,squeeze_excite,0,MiT-B0,0.582
37,SplitLawin,conv_sum_drop,G,MiT-B0,0.635
75,SplitLawin,conv_sum,G,MiT-B1,0.663
